# Training a scikit learn model on AI Platform training. 
AI Platform training can be used to train models from Tensorflow, Keras, Scikit-learn, XGBoost and Custom containers on Google Cloud Platform. 

First go to the GCP [console](https://console.cloud.google.com). When in the console navigate, using the menu on the left, to AI Platform -> Jobs. This will take you to the interface from where you will monitor your training jobs. 

Before we get started have a look at the scikit-learn model code and how the folders are structured. The code can be found in the folder: scikit-caip/trainer. 

Before you can start training the model we have to make sure that we have the required dependency installed. Run the following cells to install the needed dependencies. 

In [ ]:
!sudo pip3 install -r requirements.txt

In [ ]:
!pip freeze

In [ ]:
!sudo pip install pandas-gbq

### Train on the Cloud using AI Platform Training
If the local training works it's time to train our model on the cloud. Please change these two parameters before running the next cell:

    pathoutput gs://path_to_your_folder
    jobname <set_to_your_jobname>

Your job is submitted succesfully when you see the following:

    Job [your_jobname] submitted successfully.

Now you can go to the console to monitor your job. 

In [18]:
!gcloud ai-platform jobs submit training marketing_v1_11 \
   --staging-bucket=gs://erwinh-ml-demos \
   --region=us-central1 \
   --module-name=trainer.task \
   --package-path=trainer \
   --runtime-version 1.14 \
   --python-version 3.5 \
   -- \
   --pathdata gs://erwinh-public-data/scikit/data/marketing-data.csv \
   --pathoutput gs://erwinh-ml-demos/output-scikit/ \
   --storage GCS \
   --bqtable kfp-primer-workshop.marketing_data.raw

ERROR: (gcloud.ai-platform.jobs.submit.training) Resource in project [erwinh-ml-demos] is the subject of a conflict: Field: job.job_id Error: A job with this id already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A job with this id already exists.
    field: job.job_id


### Deploy model using AI Platform 
When the model is succesfully trained we can take the trained model and publish it as an API. Change the following variables:

    MODEL_NAME="<you_model_name>"
    MODEL_LOCATION="gs://<your_model_output_path>" 

Now check if your model is deployed. Go to the [console](https://console.cloud.google.com) -> AI Platform -> Models -> marketingpredictor. Here you will find your model. This can take a few minutes. 

In [16]:
!gcloud ai-platform models create marketingpredictor

Created ml engine model [projects/erwinh-ml-demos/models/marketingpredictor].


### Deploy the first version of you model

We are going to train a model using Google Cloud AI Platform. have a look at how the code is structured. 

To do:
+ Write the command to deploy the trained model artifact to Google AI Platform. 
+ Have a ook at the [documentation](https://cloud.google.com/ml-engine/docs/deploying-models#create_a_model_resource). 
+ Hint use gcloud: gcloud ai-platform versions create.
+ Don't forget to start your gcloud command with !

In [19]:
!gcloud ai-platform versions create marketing_v1 \
--model marketingpredictor \
--origin gs://erwinh-ml-demos/output-scikit/model \
--runtime-version 1.14 \
--framework scikit-learn \
--python-version 3.5

Creating version (this might take a few minutes)......done.                    


## Getting a prediction
After deploying our model we can call the API to get a prediction. 

To do:
+ Write to gcloud command to get a prediction.
+ Have a ook at the [documentation](https://cloud.google.com/ml-engine/docs/deploying-models#create_a_model_resource).
+ Don't forget to start your command with !

After running the cell you should see something like this:

    [True, True]

In [20]:
!gcloud ai-platform predict \
  --model marketingpredictor \
  --version marketing_v1 \
  --json-instances trainer/predict.json

[False, True]


Copyright 2019 Google Inc. All Rights Reserved. # Licensed under the Apache License, Version 2.0 (the "License"); # you may not use this file except in compliance with the License. # You may obtain a copy of the License at # # http://www.apache.org/licenses/LICENSE-2.0 # # Unless required by applicable law or agreed to in writing, software # distributed under the License is distributed on an "AS IS" BASIS, # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. # See the License for the specific language governing permissions and # limitations under the License.